<h1 align="center">An Introduction to Machine Learning - 25737</h1>
<h4 align="center">Dr. Yasaei</h4>
<h4 align="center">Sharif University of Technology, Autumn 2024</h4>

**Student Name**: Arian Bastani

**Student ID**: 400100073

# Gaussian Mixture Models with EM

## Introduction and Purpose

In this exercise, you will:

1. Implement a **Gaussian Mixture Model (GMM)** using the Expectation-Maximization (EM) algorithm **from scratch** (using NumPy and basic Python operations).
2. Implement the **same GMM model using PyTorch**.
3. Compare and contrast the two implementations (performance, complexity, ease of coding, etc.).

**Gaussian Mixture Models** assume that data is generated from a mixture of several Gaussian distributions. The EM algorithm iteratively updates the parameters (means, covariances, and mixture weights) of these Gaussians to maximize the likelihood of observed data.



## Part 1: Data Loading and Exploration

**Tasks:**  
- Load the Iris dataset and store the features in `X` and labels in `y`.
- Print the shape of `X` and examine a few rows.
- **Hint:** Use `sklearn.datasets.load_iris()` to load the data.

In [ ]:
# TODO: Load the Iris dataset and print shape
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
y = iris.target
print("Shape of X:", X.shape)
print("First 5 samples:\n", X[:5])

Shape of X: (150, 4)
First 5 samples:
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


## Part 2: Data Preprocessing (Scaling)

**Tasks:**  
- Scale the data using `StandardScaler` so that each feature has zero mean and unit variance.
- **Hint:** `from sklearn.preprocessing import StandardScaler`.


In [ ]:
# TODO: Scale the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Mean after scaling:", X_scaled.mean(axis=0))
print("Std after scaling:", X_scaled.std(axis=0))


Mean after scaling: [-1.69031455e-15 -1.84297022e-15 -1.69864123e-15 -1.40924309e-15]
Std after scaling: [1. 1. 1. 1.]


## Part 3: Implementing GMM with EM **from scratch** (NumPy-based)

We will first implement GMM using NumPy arrays and basic operations, without PyTorch.

**Tasks:**  
- Choose the number of components `K` (e.g., K=3).
- Initialize the parameters: means, covariances (diagonal), and mixture weights.
- Write functions for the E-step and M-step of the EM algorithm.
- Run the EM algorithm for a fixed number of iterations.

**Hints for Implementation:**

- Means: K x D array.
- Covariances: K x D x D (diagonal only, so you mainly store variances per feature).
- Weights: K-dimensional array, summing to 1.
- To compute Gaussian densities, recall the formula for the probability density of a multivariate Gaussian.
- For the E-step, compute responsibilities using the mixture components and their densities.
- For the M-step, update means, covariances, and weights using the responsibilities.

After implementing and running EM, extract cluster assignments by taking `argmax` of responsibilities.


In [ ]:
import numpy as np

# Set number of components
K = 3
N, D = X_scaled.shape

# TODO: Initialize means, covariances, and weights
means = np.random.rand(K, D)
covariances = 10*np.ones((K, D))  # Diagonal covariances, so you can store just var per component and construct diag
weights = np.random.rand(K)
weights /= weights.sum()

# TODO: Define a function to compute Gaussian PDF values for each component
def gaussian_pdf(X, mean, cov):
    P = (X-mean)**2 / cov.diagonal() / 2
    return np.exp(-P.sum(axis=1)) / np.sqrt(cov.diagonal().prod()) / (2*np.pi)**(mean.size/2)

# TODO: E-step
def e_step(X, means, covariances, weights):
    # Compute responsibilities
    global K, N
    r = np.zeros((N, K))
    for i in range(K):
        r[:, i] = weights[i] * gaussian_pdf(X, means[i], np.diag(covariances[i]))
    r = r / r.sum(axis=1)[:, np.newaxis]
    return r


# TODO: M-step
def m_step(X, r):
    # Update means, covariances, weights
    global N, K, D
    global weights, means, covariances
    weights = r.sum(axis=0)/N
    means = (r / r.sum(axis=0)[np.newaxis, :]).T @ X
    for i in range(K):
        covariances[i] = np.diag(((X-means[i]).T @ ((r[:, i])[:, np.newaxis] * (X-means[i]))) / np.sum(r[:, i]))

for i in range(1000):
    r = e_step(X_scaled, means, covariances, weights)
    m_step(X_scaled, r)

cluster_labels_numpy = np.argmax(r, axis=1) # argmax of responsibilities

## Part 4: Implementing GMM with EM **using PyTorch**

Now, we will implement the same algorithm using PyTorch tensors. The steps are similar, but you will use `torch` operations. This might simplify certain operations and open the door to GPU acceleration.

**Tasks:**  
- Convert `X_scaled` to a PyTorch tensor.
- Initialize parameters as `torch.tensor`s.
- Implement E-step and M-step in PyTorch.
- Run EM for a fixed number of iterations.
- Extract cluster labels.

**Hints:**
- Use `torch.tensor(X_scaled, dtype=torch.float32)` to create a PyTorch tensor.
- Operations are similar but use `torch.sum`, `torch.exp`, etc.
- Watch out for broadcasting rules and ensure shapes align.


In [ ]:
import torch

# TODO: Convert data to torch tensor
X_torch = torch.tensor(X_scaled, dtype=torch.float32)

# TODO: Initialize means, covariances, weights as torch tensors
means_torch = torch.rand((K, D))
covariances_torch = torch.ones((K, D))
weights_torch = torch.rand(K)
weights_torch /= weights_torch.sum()

# TODO: Implement gaussian_pdf using torch operations
def gaussian_pdf_torch(X, mean, cov):
    P = (X-mean)**2 / torch.diag(cov) / 2
    return torch.exp(-P.sum(axis=1)) / torch.sqrt(torch.diag(cov).prod()) / torch.pow(2*torch.pi, mean.prod()/2)

# TODO: E-step in torch
def e_step_torch(X, means, covariances, weights):
    global K, N
    r = torch.zeros((N, K))
    for i in range(K):
        r[:, i] = weights[i] * gaussian_pdf_torch(X, means[i], torch.diag(covariances[i]))
    r = r / r.sum(dim=1, keepdim=True)
    return r

# TODO: M-step in torch
def m_step_torch(X, r):
    global N, K, D
    global weights_torch, means_torch, covariances_torch
    weights_torch = r.sum(axis=0)/N
    means_torch = (r / r.sum(dim=0, keepdim=True)).T @ X
    for i in range(K):
        covariances_torch[i] = torch.diag(((X-means_torch[i]).T @ (r[:, i].unsqueeze(1) * (X-means_torch[i]))) / r[:, i].sum())


# Run EM in torch
for i in range(1000):
    r = e_step_torch(X_torch, means_torch, covariances_torch, weights_torch)
    m_step_torch(X_torch, r)

cluster_labels_torch = torch.argmax(r, axis=1) # argmax of responsibilities


## Part 5: Evaluating and Comparing Both Implementations

**Tasks:**  
- Use `adjusted_rand_score` to compare the cluster labels from both methods against the true labels `y`.
- Print the ARI for both NumPy and PyTorch implementations.
- Visually inspect if both implementations yield similar results.

**Questions:**
- Are the ARI scores similar or different between the two implementations?
- Which code was easier to write and maintain?
- Which implementation might be easier to extend to more complex models?


In [ ]:
from sklearn.metrics import adjusted_rand_score

# TODO: Compute ARI for numpy-based clustering
ari_numpy = adjusted_rand_score(y, cluster_labels_numpy)
print("ARI (NumPy):", ari_numpy)

# TODO: Compute ARI for torch-based clustering
ari_torch = adjusted_rand_score(y, cluster_labels_torch)
print("ARI (PyTorch):", ari_torch)


ARI (NumPy): 0.8342589385759397
ARI (PyTorch): 0.8022085453675192


**Questions:**  
1. **Implementation Detail:** What are the main differences in code complexity between a plain NumPy-based implementation and a PyTorch-based one?  
answer:

2. **Performance:** Which implementation is likely to be more efficient or easier to parallelize and why?  
answer:
3. **Numerical Stability:** How might PyTorch’s built-in functions improve numerical stability compared to a manual implementation?  
answer:

4. **Extendability:** If you wanted to add more complex features (e.g., full covariance matrices, regularization), which approach would be simpler and why?
answer:

**1.** NumPy requires more manual coding of math operations; PyTorch provides higher-level functions and automatic differentiation, making complex models easier to implement.

**2.** PyTorch is generally more efficient due to GPU acceleration, automatic parallelization, and optimized operations.

**3.** PyTorch offers built-in functions and optimized implementations that improve numerical stability compared to manual NumPy implementations.


**4.** PyTorch's automatic differentiation and modular design make it much simpler to add complex features.
